# K-Means classification

K-means clustering is one of the basic unsupervised classification algorithms. Its objective is to group similar data points together (clusters), by following patterns. The first step is to define a number k, which represents the number of clusters that will be formed. The center of each cluster is called “centroid”. When analyzing data, the algorithm identifies k centroids and allocates each data point to the nearest cluster.
At the beginning, the algorithm starts with a first group of random centroids, then it performs repetitive calculations to optimize their positions. It stops whether the pre-set number of iterations has been reached or if there is no change in the positions of the centers.
In order to determine the In case of natural language processing, word2vec embeddings are used in order to determine the position of each word in the multidimensional space. 
The algorithm consists of several steps. Firstly, the corpus is transformed into array of real numbers and data is positioned in a multi-dimensional space. Then, two phases are repeated for a fixed number of iterations: cluster assignment and centroid move. The algorithm goes through each of the data points and depending on which cluster is closer, it assigns the data point to it. It then calculates the average of all the points in a cluster and moves the centroid to that average location.


In [ ]:
%run "NLP_clustering.ipynb"

# the second method: starting from the preprocessed title + metadata abstract
# you can use k-means with a variable number of clusters to get text classification
# and check the optimal number of clusters using 2 methods

if __name__ == "__main__":

    csw = CatalogueServiceWeb('http://geocatalog.webservice-energy.org/geonetwork/srv/eng/csw')
    
    set_title = fes.PropertyIsLike('any', '')
    filter_list = [set_title]

    csw.getrecords2(constraints=filter_list, maxrecords=2000)

    fmt = '{:*^64}'.format
    print(fmt(' Catalog information '))
    print("CSW version: {}".format(csw.version))
    print("Number of datasets available: {}".format(len(csw.records.keys())))
    print('\n')

    original_list_of_titles = []
    preprocessed_list_of_titles = []
    word2vec_number_list = []
    
    model, index2word_set = init_language_model()
    
    for rec in csw.records:
        original_list_of_titles.append(csw.records[rec].title)
        title = prepareDescription(csw.records[rec].title, keepwords)
        if csw.records[rec].abstract != '':
            abstract = prepareDescription(csw.records[rec].abstract, keepwords)
            title = title + " " + abstract
        
        preprocessed_list_of_titles.append(title.split())

    model = gensim.models.Word2Vec(preprocessed_list_of_titles, min_count = 1, size=32)
    
    X=[]
    for sentence in preprocessed_list_of_titles:
        X.append(sent_vectorizer(sentence, model))

    # Run the K-means algorithm for a variable nunbers of cluster
    # in order to get the optimal value
    K = range(2, 21)
    distortions = []
    silhs = []
    
    max_k_silh = 1
    max_k_cal = 1
    max_silh = -1
    max_cal = -1
    
    for k in K:
        kclusterer = KMeansClusterer(k, distance=nltk.cluster.util.euclidean_distance, repeats=25, avoid_empty_clusters=True)
        assigned_clusters = kclusterer.cluster(X, assign_clusters=True)

        kmeans = cluster.KMeans(n_clusters=k)
        kmeans.fit(X)

        labels = kmeans.labels_
        centroids = kmeans.cluster_centers_
        
        print ("Cluster id labels for inputted data")
        print (labels)
        print ("Centroids data")
        print (centroids)

        print ("Score (Opposite of the value of X on the K-means objective which is Sum of distances of samples to their closest cluster center):")
        print (kmeans.score(X))

        silhouette_score = metrics.silhouette_score(X, labels, metric='euclidean')
        
        print ("Silhouette_score: ")
        print (silhouette_score)
        
        if (silhouette_score > max_silh):
            max_silh = silhouette_score
            max_k_silh = k
            
        harabaz_score = metrics.calinski_harabaz_score(X, labels)
        if harabaz_score > max_cal:
            max_cal = harabaz_score
            max_k_cal = k

        distortions.append(kmeans.inertia_)
        silhs.append(silhouette_score)
        
        import matplotlib.pyplot as plt

        from sklearn.manifold import TSNE

        model = TSNE(n_components=2, random_state=0)
        np.set_printoptions(suppress=True)

        Y=model.fit_transform(X)


        plt.scatter(Y[:, 0], Y[:, 1], c=assigned_clusters, s=290,alpha=.5)


        for j in range(len(preprocessed_list_of_titles)):    
            plt.annotate(assigned_clusters[j],xy=(Y[j][0], Y[j][1]),xytext=(0,0),textcoords='offset points')
           #print ("%s %s" % (assigned_clusters[j],  preprocessed_list_of_titles[j]))


        plt.show()
    
    """
    Plot the distortions for the elbow method
    
    This method looks at the percentage of variance explained as a function of the number of clusters:
    One should choose a number of clusters so that adding another cluster doesn't give much better modeling
    of the data. More precisely, if one plots the percentage of variance explained by the clusters against the
    number of clusters, the first clusters will add much information (explain a lot of variance), but at some point
    the marginal gain will drop, giving an angle in the graph.
    This is a visual solution, but it is often ambiguous and not very reliable.
    """
    plt.plot(K, distortions, 'bx-')
    plt.xlabel('k')
    plt.ylabel('Distortion')
    plt.title('The Elbow Method showing the optimal k')
    plt.show()

    """
    A better approach is the silhoutte, which provides a succinct graphical representation of how well each
    object has been classified.
    The silhouette value  is a measure of how similar an object is to its own cluster (cohesion) compared to other
    clusters (separation). The silhouette ranges from −1 to +1, where a high value indicates that the object is well
    matched to its own cluster and poorly matched to neighboring clusters. If most objects have a high value, then
    the clustering configuration is appropriate.
    """
    plt.plot(K, silhs, 'bx-')
    plt.xlabel('k')
    plt.ylabel('Distortion')
    plt.title('Silhouette score')
    plt.show()
    
    print('============================================================')
    print('k max for silhouette: %d' %max_k_silh)
    print('k max for calinski_harabaz_score: %d' %max_k_cal)
    print('============================================================')

 
    """
    print ("Cluster id labels for inputted data")
    print (labels)
    print ("Centroids data")
    print (centroids)

    print ("Score (Opposite of the value of X on the K-means objective which is Sum of distances of samples to their closest cluster center):")
    print (kmeans.score(X))

    silhouette_score = metrics.silhouette_score(X, labels, metric='euclidean')

    print ("Silhouette_score: ")
    print (silhouette_score)


    import matplotlib.pyplot as plt

    from sklearn.manifold import TSNE

    model = TSNE(n_components=NUM_CLUSTERS, random_state=0)
    np.set_printoptions(suppress=True)

    Y=model.fit_transform(X)


    plt.scatter(Y[:, 0], Y[:, 1], c=assigned_clusters, s=290,alpha=.5)


    for j in range(len(preprocessed_list_of_titles)):    
       plt.annotate(assigned_clusters[j],xy=(Y[j][0], Y[j][1]),xytext=(0,0),textcoords='offset points')
       print ("%s %s" % (assigned_clusters[j],  preprocessed_list_of_titles[j]))


    plt.show()

    word_vectors = model.wv.syn0
    num_clusters = 4
    kmeans_clustering = KMeans( n_clusters = num_clusters )
    idx = kmeans_clustering.fit_predict( word_vectors )
    word_centroid_map = dict(zip( model.wv.index2word, idx ))
    
    for cluster in range(0,num_clusters):
        print("\nCluster %d" % cluster)
        # Find all of the words for that cluster number, and print them out
        words = []
        v = list(word_centroid_map.values())
        for i in range(len(v)):
            if(v[i] == cluster ):mes
                k = list(word_centroid_map.keys()) 
                words.append(k[i])
        print(words)
        """

In [ ]:
%run "NLP_clustering.ipynb"

from gensim.corpora import Dictionary
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from gensim.models import Phrases

#based on k-means, you can get the optimal number of clusters based on the previous information
#starting from it, using the metadata records already classified, you can determine the most important
#words in each topic, using the bag of words approach
if __name__ == "__main__":

    csw = CatalogueServiceWeb('http://geocatalog.webservice-energy.org/geonetwork/srv/eng/csw')
    
    set_title = fes.PropertyIsLike('any', '')#SEARCH_QUERY)
    filter_list = [set_title]

    csw.getrecords2(constraints=filter_list, maxrecords=2000)

    fmt = '{:*^64}'.format
    print(fmt(' Catalog information '))
    print("CSW version: {}".format(csw.version))
    print("Number of datasets available: {}".format(len(csw.records.keys())))
    print('\n')

    original_list_of_titles = []
    preprocessed_list_of_titles = []
    identifiers = []
    word2vec_number_list = []
    
    for rec in csw.records:
        title = prepareDescription(csw.records[rec].title, keepwords)
        identifiers.append(csw.records[rec].identifier)
        if csw.records[rec].abstract != '':
            abstract = prepareDescription(csw.records[rec].abstract, keepwords)
            title = title + " " + abstract
        
        preprocessed_list_of_titles.append(word_tokenize(title))


    assigned_clusters = [-1] * len(preprocessed_list_of_titles)
    
    while 1:
        model = gensim.models.Word2Vec(preprocessed_list_of_titles, min_count = 1, size=32)

        X=[]
        for sentence in preprocessed_list_of_titles:
            X.append(sent_vectorizer(sentence, model))

        k = 2
        kclusterer = KMeansClusterer(k, distance=nltk.cluster.util.cosine_distance, repeats=25, avoid_empty_clusters=True)
        assigned_clusters = kclusterer.cluster(X, assign_clusters=True)

        kmeans = cluster.KMeans(n_clusters=k)
        kmeans.fit(X)

        labels = kmeans.labels_
        centroids = kmeans.cluster_centers_

        model = TSNE(n_components=2, random_state=0)
        np.set_printoptions(suppress=True)

        Y=model.fit_transform(X)

        plt.scatter(Y[:, 0], Y[:, 1], c=assigned_clusters, s=290,alpha=.5)


        for j in range(len(preprocessed_list_of_titles)):
            plt.annotate(assigned_clusters[j],xy=(Y[j][0], Y[j][1]),xytext=(0,0),textcoords='offset points')


        plt.show()

        model, index2word_set = init_language_model()

        #obtain a list of words from the list of titles + abstracts
        no_of_docs = [0 for i in range(k)]
        for i in range(k):
            documents = []

            for j in range(len(preprocessed_list_of_titles)):
                if assigned_clusters[j] == i:
                    no_of_docs[i] += 1
                    documents.append(preprocessed_list_of_titles[j])

            bigram = gensim.models.Phrases(documents, min_count=5, threshold=100)
            bigram_mod = gensim.models.phrases.Phraser(bigram)
            data_words_bigrams = [bigram_mod[doc] for doc in documents]

            # Do lemmatization keeping only noun, adjectives, verbs and adverbs
            nlp = spacy.load('en', disable=['parser', 'ner'])
            data_lemmatized = lemmatization(data_words_bigrams, nlp, ['NOUN', 'ADJ', 'VERB', 'ADV'])
            dictionary = Dictionary(data_lemmatized)
            #dictionary = Dictionary(documents)

            bow_corpus = [dictionary.doc2bow(doc) for doc in documents]

            lda_model =  gensim.models.LdaMulticore(bow_corpus,
                                           num_topics = 1,
                                           id2word = dictionary,
                                           passes = 20,
                                           workers = 2)
            #pprint(lda_model.print_topics())
            print(lda_model.show_topic(0, topn=10))

     #       doc_lda = lda_model[bow_corpus]

        for i in range(k):
            print("-------------------------------- cluster number %d" %i)
            print("cluster " + str(i) + " contains " + str(no_of_docs[i]) + " documents")
            count = 0
            for j in range(len(preprocessed_list_of_titles)):
                if assigned_clusters[j] == i and count < 10:
                    print("http://geocatalog.webservice-energy.org/geonetwork/srv/eng/csw?REQUEST=GetRecordById&id=" + str(identifiers[j]) + "&SERVICE=CSW&VERSION=2.0.2")
                    count += 1
                if count >= 10:
                    break

        split_again = input("Do you want to get further categories [Y/n] ")
        if split_again == 'n':
            break
        new_list_of_titles = []
        categ = input("Which one do you want to get [1/2] ")
        for j in range(len(preprocessed_list_of_titles)):
            if assigned_clusters[j] == int(categ) - 1:
                new_list_of_titles.append(preprocessed_list_of_titles[j])
        preprocessed_list_of_titles = new_list_of_titles
        
        

In [ ]:
#based on k-means, you can get the optimal number of clusters based on the previous information
#starting from it, use directly the LDA approch to divide articles into topics and determine the most important
#words in each topic, using the bag of words approach

from itertools import chain


if __name__ == "__main__":

    csw = CatalogueServiceWeb('http://geocatalog.webservice-energy.org/geonetwork/srv/eng/csw')
    set_title = fes.PropertyIsLike('any', '')#SEARCH_QUERY)
    filter_list = [set_title]

    csw.getrecords2(constraints=filter_list, maxrecords=2000)

    fmt = '{:*^64}'.format
    print(fmt(' Catalog information '))
    print("CSW version: {}".format(csw.version))
    print("Number of datasets available: {}".format(len(csw.records.keys())))
    print('\n')

    original_list_of_titles = []
    preprocessed_list_of_titles = []
    identifiers = []
    word2vec_number_list = []
    
    model, index2word_set = init_language_model()
    
    for rec in csw.records:
        original_list_of_titles.append(csw.records[rec].title)
        identifiers.append(csw.records[rec].identifier)
        title = prepareDescription(csw.records[rec].title, keepwords)
        if csw.records[rec].abstract != '':
            abstract = prepareDescription(csw.records[rec].abstract, keepwords)
            title = title + " " + abstract
        
        #sent = avg_feature_vector(keyw, model, num_features=NUM_FEATURES, index2word_set=index2word_set)
        #if len(title) != 0:
        preprocessed_list_of_titles.append(title.split())
        #word2vec_number_list.append(sent)

    k = 4
    documents = []
    
    #preprocessed_list_of_titles = lemmatization()
    
    for j in range(len(preprocessed_list_of_titles)):
        documents.append(preprocessed_list_of_titles[j])

    # Build the bigram and trigram models
    #bigram = gensim.models.Phrases(documents, min_count=5, threshold=100) # higher threshold fewer phrases.
    #trigram = gensim.models.Phrases(bigram[documents], threshold=100)

    # Faster way to get a sentence clubbed as a trigram/bigram
    #bigram_mod = gensim.models.phrases.Phraser(bigram)
    #trigram_mod = gensim.models.phrases.Phraser(trigram)

    #data_words_bigrams = [bigram_mod[doc] for doc in documents]
    
    bow_corpus = [dictionary.doc2bow(doc) for doc in documents]
    tfidf = gensim.models.TfidfModel(bow_corpus)
    corpus_tfidf = tfidf[bow_corpus]
    lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=k, id2word=dictionary, passes=2, workers=4)
    
    """
    lda_model =  gensim.models.LdaMulticore(bow_corpus,
                                       num_topics = k,
                                       id2word = dictionary,
                                       passes = 50,
                                       workers = 2)
    """
    for idx, topic in lda_model.print_topics(-1):
        print('Topic: {} Word: {}'.format(idx, topic))
    
    #pprint(lda_model.show_topics())
    
    lda_corpus = lda_model[bow_corpus]
    scores = list(chain(*[[score for topic_id,score in topic] \
                      for topic in [doc for doc in lda_corpus]]))
    threshold = sum(scores)/len(scores)
    print(threshold)
    print()
    
    no_of_docs = [0] * k
    assigned_clusters = [-1] * len(documents)
    
    for id in range(k):
        no = 0
        for i,j in zip(lda_corpus,documents):
            #print(str(no) + "-------------------------------" + str(i))
            if len(i) > id:
                if i[id][1] > threshold:
                    assigned_clusters[no] = id
                    no_of_docs[id] += 1
            no += 1
    
    
    for i in range(k):
        print("cluster number " + str(i) + " has " + str(no_of_docs[i]) +" entries")
        count = 0
        for j in range(len(preprocessed_list_of_titles)):
            if assigned_clusters[j] == i and count < 10:
                print("http://geocatalog.webservice-energy.org/geonetwork/srv/eng/csw?REQUEST=GetRecordById&id=" + str(identifiers[j]) + "&SERVICE=CSW&VERSION=2.0.2")
                count += 1
            if count >= 10:
                break

    """
    for i in range(k):
        print("-------------------------------- cluster number %d" %i)
        count = 0
        for j in range(len(preprocessed_list_of_titles)):
            if assigned_clusters[j] == i and count < 10:
                print("http://geocatalog.webservice-energy.org/geonetwork/srv/eng/csw?REQUEST=GetRecordById&id=" + str(identifiers[j]) + "&SERVICE=CSW&VERSION=2.0.2")
                count += 1
            if count >= 10:
                break
    """


In [ ]:
# ========= try again other techniques ===========

#based on k-means, you can get the optimal number of clusters based on the previous information
#starting from it, use directly the LDA approch to divide articles into topics and determine the most important
#words in each topic, using the bag of words approach

from itertools import chain
import pandas as pd

def topic_prob_extractor(gensim_hdp):
    shown_topics = gensim_hdp.show_topics(num_topics=-1, formatted=False)
    topics_nos = [x[0] for x in shown_topics ]
    weights = [ sum([item[1] for item in shown_topics[topicN][1]]) for topicN in topics_nos ]

    return pd.DataFrame({'topic_id' : topics_nos, 'weight' : weights})

def evaluate_graph(dictionary, corpus, texts, limit):
    """
    Function to display num_topics - LDA graph using c_v coherence
    
    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    limit : topic limit
    
    Returns:
    -------
    lm_list : List of LDA topic models
    c_v : Coherence values corresponding to the LDA model with respective number of topics
    """
    c_v = []
    lm_list = []
    for num_topics in range(1, limit):
        lm = gensim.models.LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)
        lm_list.append(lm)
        cm = gensim.models.CoherenceModel(model=lm, texts=texts, dictionary=dictionary, coherence='c_v')
        c_v.append(cm.get_coherence())
        
    # Show graph
    x = range(1, limit)
    plt.plot(x, c_v)
    plt.xlabel("num_topics")
    plt.ylabel("Coherence score")
    plt.legend(("c_v"), loc='best')
    plt.show()
    
    return lm_list, c_v



if __name__ == "__main__":

    csw = CatalogueServiceWeb('http://geocatalog.webservice-energy.org/geonetwork/srv/eng/csw')
    set_title = fes.PropertyIsLike('any', '')#SEARCH_QUERY)
    filter_list = [set_title]

    csw.getrecords2(constraints=filter_list, maxrecords=2000)

    fmt = '{:*^64}'.format
    print(fmt(' Catalog information '))
    print("CSW version: {}".format(csw.version))
    print("Number of datasets available: {}".format(len(csw.records.keys())))
    print('\n')

    original_list_of_titles = []
    preprocessed_list_of_titles = []
    identifiers = []
    word2vec_number_list = []
    
    model, index2word_set = init_language_model()
    
    for rec in csw.records:
        original_list_of_titles.append(csw.records[rec].title)
        identifiers.append(csw.records[rec].identifier)
        title = prepareDescription(csw.records[rec].title, keepwords)
        if csw.records[rec].abstract != '':
            abstract = prepareDescription(csw.records[rec].abstract, keepwords)
            title = title + " " + abstract
        
        #sent = avg_feature_vector(keyw, model, num_features=NUM_FEATURES, index2word_set=index2word_set)
        #if len(title) != 0:
        preprocessed_list_of_titles.append(title.split())
        #word2vec_number_list.append(sent)

    nlp = spacy.load('en', disable=['parser', 'ner'])
    preprocessed_list_of_titles = lemmatization(preprocessed_list_of_titles, nlp)
    bow_corpus = [dictionary.doc2bow(doc) for doc in preprocessed_list_of_titles]
    
    
    """
    k = 7
    documents = []
    
    for j in range(len(preprocessed_list_of_titles)):
        documents.append(preprocessed_list_of_titles[j])

    # Build the bigram and trigram models
    bigram = gensim.models.Phrases(documents, min_count=5, threshold=100) # higher threshold fewer phrases.
    trigram = gensim.models.Phrases(bigram[documents], threshold=100)

    # Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)

    data_words_bigrams = [bigram_mod[doc] for doc in documents]
    
    bow_corpus = [dictionary.doc2bow(doc) for doc in documents]
    
    lda_model =  gensim.models.LsiModel(bow_corpus, num_topics = 7,
                                       id2word = dictionary)
    
    print(lda_model.show_topics(num_topics=7))
    """
    
    lmlist, c_v = evaluate_graph(dictionary=dictionary, corpus=bow_corpus, texts=preprocessed_list_of_titles, limit=20)
    
    """
    bow_corpus = [dictionary.doc2bow(doc) for doc in documents]
    lda_model =  gensim.models.HdpModel(bow_corpus,
                                       id2word = dictionary)
    
    data_frame = topic_prob_extractor(lda_model)
    data_frame = data_frame.sort_values(by='weight', ascending=False)
    print(data_frame)


    pprint(lda_model.print_topics(-1, 10))
    
  
    lda_corpus = lda_model[bow_corpus]
    scores = list(chain(*[[score for topic_id,score in topic] \
                      for topic in [doc for doc in lda_corpus]]))
    threshold = sum(scores)/len(scores)
    print(threshold)
    print()
    
    k = 150
    
    for id in range(k):
        no = 0
        for i,j in zip(lda_corpus,documents):
            pprint(i)
            if i != []:
                if i[id][1] > threshold:
                    assigned_clusters[no] = id
                no += 1
    
    for i in range(k):
        print("-------------------------------- cluster number %d" %i)
        count = 0
        for j in range(len(preprocessed_list_of_titles)):
            if assigned_clusters[j] == i and count < 10:
                print("http://geocatalog.webservice-energy.org/geonetwork/srv/eng/csw?REQUEST=GetRecordById&id=" + str(identifiers[j]) + "&SERVICE=CSW&VERSION=2.0.2")
                count += 1
            if count >= 10:
                break
    """    

In [ ]:
#based on k-means, you can get the optimal number of clusters based on the previous information
#starting from it, use directly the LDA approch to divide articles into topics and determine the most important
#words in each topic, using the bag of words approach

%run "NLP_clustering.ipynb"

from itertools import chain
# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint
import gensim.corpora as corpora

from gensim.models import CoherenceModel
import operator

import pandas as pd


# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline


if __name__ == "__main__":

    csw = CatalogueServiceWeb('http://geocatalog.webservice-energy.org/geonetwork/srv/eng/csw')
    set_title = fes.PropertyIsLike('any', '')#SEARCH_QUERY)
    filter_list = [set_title]

    csw.getrecords2(constraints=filter_list, maxrecords=2000)

    fmt = '{:*^64}'.format
    print(fmt(' Catalog information '))
    print("CSW version: {}".format(csw.version))
    print("Number of datasets available: {}".format(len(csw.records.keys())))
    print('\n')

    original_list_of_titles = []
    preprocessed_list_of_titles = []
    identifiers = []
    word2vec_number_list = []
    
    #model, index2word_set = init_language_model()
    
    print("------------------------------------------------------")
    data = []
    for rec in csw.records:
        original_list_of_titles.append(csw.records[rec].title)
        identifiers.append(csw.records[rec].identifier)
        title = prepareDescription(csw.records[rec].title, keepwords)
        if csw.records[rec].abstract != '':
            abstract = prepareDescription(csw.records[rec].abstract, keepwords)
            title = title + " " + abstract
        
        #sent = avg_feature_vector(keyw, model, num_features=NUM_FEATURES, index2word_set=index2word_set)
        #if len(title) != 0:
        preprocessed_list_of_titles.append(title.split())
        data.append(title)
        #word2vec_number_list.append(sent)
        break
   
    # Build the bigram and trigram models
    bigram = gensim.models.Phrases(preprocessed_list_of_titles, min_count=5, threshold=32) # higher threshold fewer phrases.
    trigram = gensim.models.Phrases(bigram[preprocessed_list_of_titles], threshold=32)  

    # Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    
    print(trigram_mod[bigram_mod[preprocessed_list_of_titles[0]]])
    
    # Form Bigrams
    data_words_bigrams = make_bigrams(preprocessed_list_of_titles)

    # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
    # python3 -m spacy download en
    nlp = spacy.load('en', disable=['parser', 'ner'])

    # Do lemmatization keeping only noun, adj, vb, adv
    data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    
    # Create Dictionary
    id2word = corpora.Dictionary(data_lemmatized)

    # Create Corpus
    texts = data_lemmatized

    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in texts]
    
    k = 5
    
    # Build LDA model
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=k, 
                                           random_state=0,
                                           update_every=1,
                                           chunksize=5,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
    
    pprint(lda_model.print_topics())
    doc_lda = lda_model[corpus]
    
    # Compute Perplexity
    print ('Perplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

    # Compute Coherence Score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print ('Coherence Score: ', coherence_lda)
    
    # Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
    mallet_path = 'mallet-2.0.8/bin/mallet' # update this path
    ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)
    
    # Show Topics
    pprint(ldamallet.show_topics(num_topics=k, formatted=False))

    # Compute Coherence Score
    coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    coherence_ldamallet = coherence_model_ldamallet.get_coherence()
    print('\nCoherence Score: ', coherence_ldamallet)
    
    """
    # Can take a long time to run.
    limit=35; start=2; step=1;
    model_list, coherence_values = compute_coherence_values(dictionary=id2word,
                                                        corpus=corpus,
                                                        texts=data_lemmatized,
                                                        start=start,
                                                        limit=limit,
                                                        step=step)
    
    # Show graph
    x = range(start, limit, step)
    plt.figure(figsize=(15, 10))
    plt.plot(x, coherence_values)
    plt.xlabel("Num Topics")
    plt.ylabel("Coherence score")
    # plt.legend(("coherence_values"), loc='best')
    plt.show()
    
    # Print the coherence scores
    for m, cv in zip(x, coherence_values):
        print("Num Topics =", m, " has Coherence Value of", round(cv, 6))
        
    # Select the model and print the topics
    index, value = max(enumerate(coherence_values), key=operator.itemgetter(1))
    optimal_model = model_list[index]
    k = index + 1
    """
    optimal_model = lda_model
    model_topics = optimal_model.show_topics(num_topics=k, formatted=False)
    pprint(optimal_model.print_topics(num_words=10))
    
    optimal_model.show_topic(0,10)
    for topic in sorted(optimal_model.show_topics(num_topics=1000, num_words=10, formatted=False), key=lambda x: x[0]):
        print('Topic {}: {}'.format(topic[0], [item[0] for item in topic[1]]))

        
    topics = lda_model.get_document_topics(corpus, per_word_topics=True)
    all_topics = [(doc_topics, word_topics, word_phis) for doc_topics, word_topics, word_phis in topics]
    
    """    
    
    lda_corpus = lda_model[corpus]
    scores = list(chain(*[[score for topic_id,score in topic] \
                      for topic in [doc for doc in lda_corpus]]))
    threshold = sum(scores)/len(scores)
    print(threshold)
    print()
    
    no_of_docs = [0] * k
    assigned_clusters = [-1] * len(documents)
    
    for id in range(k):
        no = 0
        for i,j in zip(lda_corpus,documents):
            #print(str(no) + "-------------------------------" + str(i))
            if len(i) > id:
                if i[id][1] > threshold:
                    assigned_clusters[no] = id
                    no_of_docs[id] += 1
            no += 1
    
    
    for i in range(k):
        print("cluster number " + str(i) + " has " + str(no_of_docs[i]) +" entries")
        count = 0
        for j in range(len(preprocessed_list_of_titles)):
            if assigned_clusters[j] == i and count < 10:
                print("http://geocatalog.webservice-energy.org/geonetwork/srv/eng/csw?REQUEST=GetRecordById&id=" + str(identifiers[j]) + "&SERVICE=CSW&VERSION=2.0.2")
                count += 1
            if count >= 10:
                break
    
    
    print("-----------------------------------------------------")
    for doc in corpus:
        topics_distr = lda_model[doc]
        print(topics_distr)
        for _, topic in enumerate(topics_distr):
    """
    df_topic_sents_keywords = format_topics_sentences(texts=data, ldamodel=optimal_model, corpus=corpus)

    # Format
    df_dominant_topic = df_topic_sents_keywords.reset_index()
    df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
    
    # Show
    #print(df_dominant_topic)
    
    #print(df_dominant_topic[df_dominant_topic['Dominant_Topic'].isin([0, 1])])
    
    #print([text.split() for text in df_dominant_topic['Keywords'].tolist()])
    
    count = 0
    for i in range(k):
        print("--------------------- Topic " + str(i))
        j = 0
        count = 0
        for idx, row in df_dominant_topic.iterrows():
            #print('{}. Dominant keywords: {}'.format(row['Document_No'], row['Keywords'].split(', ')[:5]))
            if count >= 10:
                break
            if int(row['Dominant_Topic']) == i:
                print("http://geocatalog.webservice-energy.org/geonetwork/srv/eng/csw?REQUEST=GetRecordById&id=" + str(identifiers[j]) + "&SERVICE=CSW&VERSION=2.0.2")
                count += 1
            j += 1
    
    """
    # Group top 5 sentences under each topic
    sent_topics_sorteddf_mallet = pd.DataFrame()

    sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

    for i, grp in sent_topics_outdf_grpd:
        sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

    # Reset Index    
    sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

    # Format
    sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

    # Show
    print(sent_topics_sorteddf_mallet)

    """